In [1]:
import math
import numpy as np
from sklearn.preprocessing import StandardScaler, MinMaxScaler

import custom_scores
import CMAPSAuxFunctions
from tunableModels import TunableModel

from keras.models import Sequential, Model
from keras.layers import Dense, Input, Dropout, Reshape, Conv2D, Flatten, MaxPooling2D
from keras.optimizers import Adam
from keras.callbacks import LearningRateScheduler
from keras import backend as K

from scipy.optimize import differential_evolution

C:\Users\controlslab\Anaconda3\envs\tensorflow\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
def RULmodel_SN_1(input_shape):
    
    #Create a sequential model
    model = Sequential()
    
    #Add the layers for the model
    model.add(Dense(250, input_dim=input_shape, activation='relu', kernel_initializer='glorot_normal', name='fc1'))
    model.add(Dropout(0.6))
    model.add(Dense(50, activation='relu', kernel_initializer='glorot_normal', name='fc2'))
    #model.add(Dense(5, activation='relu', kernel_initializer='glorot_normal', name='fc3'))
    model.add(Dropout(0.2))
    #model.add(Dense(10, activation='relu', name='fc3'))
    #model.add(Dropout(0.5))
    #model.add(Dense(10, activation='tanh', name='fc4'))
    #model.add(Dropout(0.5))
    model.add(Dense(1, activation='linear', name='out'))
    
    #create a placeholder for the input
    #X_input = Input(shape=(input_shape))
    
    #Create the layers
    #X = Dense(100, activation='relu', name='fc1')(X_input)
    #X = Dense(100, activation='relu', name='fc2')(X)
    #X = Dense(1, activation='linear', name='out')(X)
    
    # Create model. This creates the Keras model instance, you'll use this instance to train/test the model.
    #model = Sequential(inputs = X_input, outputs = X, name='RUL')
    
    return model

In [3]:
def nn_optmize_fun(x, selected_features=['T24', 'T30', 'T50', 'P30', 'Nf', 'Nc', 'Ps30', 'phi', 'NRf', 'NRc', 
                     'BPR', 'htBleed', 'W31', 'W32'], datasetNumber = '1', scaler = None, verbose=0, epochs=250, 
                  saveToFile = None):
    
    #Clear the previous tensorflow graph
    K.clear_session()
    
    #Extract the tunning variables from the input vector
    #Round the values to the nearest integer since this implementation is for real numbers
    x = x.astype(int)
    windows_size = x[0]
    window_stride = x[1]
    constantRUL = x[2]
    
    #print("Creating model")
    #Shared parameters for the models
    optimizer = Adam(lr=0, beta_1=0.5)
    lossFunction = "mean_squared_error"
    metrics = ["mse"]
    
    #Define the model
    nFeatures = len(selected_features)
    shapeSN = nFeatures*windows_size
    modelRULSN = RULmodel_SN_1(shapeSN)
    modelRULSN.compile(optimizer = optimizer, loss = lossFunction, metrics = metrics)

    #load the data using the selected parameters
    tModel = TunableModel(selected_features, scaler = min_max_scaler, window_stride = window_stride, 
                          window_size = windows_size, constRul = constantRUL, datasetNumber = datasetNumber, 
                         epochs=epochs)
    
    tModel.loadData()
    
    #Add the models to the tunableModel object
    tModel.addModel("RUL_SN_1_optmizable", modelRULSN)
    
    #print("Training model")
    #Train the model
    lrate = LearningRateScheduler(CMAPSAuxFunctions.step_decay)
    tModel.trainCurrentModel(learningRateScheduler=lrate, verbose=verbose)
    time = tModel.getModelTimes(tModel.currentModelName)
    #print("Training time {}".format(time))
    
    #print("Assesing model performance")
    #Assess the model performance
    tModel.evaluateCurrentModel(["rhs"])
    cScores = tModel.getModelScores(tModel.currentModelName)
    rmse = math.sqrt(cScores['score_1'])
    rhs = cScores['rhs']
    #print("The score for this model is: {}".format(rmse))
    
    msgStr = "The model variables are " + str(x) + "\tThe scores are: [RMSE:{:.4f}, RHS:{:.4f}]".format(rmse, rhs)
    
    if saveToFile is not None:
        print(msgStr)
        saveToFile.write(msgStr)
    else:
        print(msgStr)
    
    #Return RMSE as the performance metric to steer the search
    return rmse
    

In [4]:
file = open("testFile.txt", "w") 

#Selected as per CNN paper
selected_features = ['T24', 'T30', 'T50', 'P30', 'Nf', 'Nc', 'Ps30', 'phi', 'NRf', 'NRc', 
                     'BPR', 'htBleed', 'W31', 'W32']

min_max_scaler = MinMaxScaler(feature_range=(-1, 1))

res = nn_optmize_fun(np.array([30, 1, 125]), selected_features = selected_features, scaler = min_max_scaler, 
                     datasetNumber = '1', verbose=1, epochs=20, saveToFile = file)

file.close()
print(res)

Instructions for updating:
keep_dims is deprecated, use keepdims instead
Epoch 1/20
17731/17731 [==============================] - 0s 22us/step - loss: 3449.7034 - mean_squared_error: 3449.7034
Epoch 2/20
17731/17731 [==============================] - 0s 9us/step - loss: 545.1789 - mean_squared_error: 545.1789
Epoch 3/20
17731/17731 [==============================] - 0s 9us/step - loss: 438.0407 - mean_squared_error: 438.0407
Epoch 4/20
17731/17731 [==============================] - 0s 10us/step - loss: 396.8468 - mean_squared_error: 396.8468
Epoch 5/20
17731/17731 [==============================] - 0s 9us/step - loss: 372.6428 - mean_squared_error: 372.6428
Epoch 6/20
17731/17731 [==============================] - 0s 8us/step - loss: 357.6581 - mean_squared_error: 357.6581
Epoch 7/20
17731/17731 [==============================] - 0s 9us/step - loss: 353.9545 - mean_squared_error: 353.9545
Epoch 8/20
17731/17731 [==============================] - 0s 8us/step - loss: 350.0189 - mean_squ

In [ ]:
#Optimize the parameters for the NN using DE

maxWindowSize = {'1':30, '2':20, '3':30, '4':30}
datasetNumber = '1'

selected_features = ['T24', 'T30', 'T50', 'P30', 'Nf', 'Nc', 'Ps30', 'phi', 'NRf', 'NRc', 
                     'BPR', 'htBleed', 'W31', 'W32']

min_max_scaler = MinMaxScaler(feature_range=(-1, 1))

windowSizeBounds = [1,maxWindowSize[datasetNumber]]
windowStrideBounds = [1,10]
constantRULBounds = [80,140]

bounds = [windowSizeBounds, windowStrideBounds, constantRULBounds]




results = differential_evolution(nn_optmize_fun, bounds, 
                                 args=(selected_features, datasetNumber, min_max_scaler, 0),
                                strategy='best1bin', maxiter=10, popsize=10, disp=True)

Creating model
Instructions for updating:
keep_dims is deprecated, use keepdims instead
Loading data for dataset 1
Training model
Training time 9.047229776926613
Assesing model performance
100/100 [==============================] - 0s 251us/step
The score for this model is: 14.935086796766637
Creating model
Loading data for dataset 1
Training model
Training time 37.584378158093315
Assesing model performance
100/100 [==============================] - 0s 311us/step
The score for this model is: 13.280083772326137
Creating model
Loading data for dataset 1
Training model
Training time 34.83250320304567
Assesing model performance
100/100 [==============================] - 0s 391us/step
The score for this model is: 15.722218666270962
Creating model
Loading data for dataset 1
Training model
Training time 7.320036376358786
Assesing model performance
100/100 [==============================] - 0s 472us/step
The score for this model is: 19.733076044072114
Creating model
Loading data for dataset 1


Training model
Training time 6.4035730263935875
Assesing model performance
100/100 [==============================] - 0s 4ms/step
The score for this model is: 15.575528224358024
Creating model
Loading data for dataset 1
Training model
Training time 6.583385610126811
Assesing model performance
100/100 [==============================] - 0s 4ms/step
The score for this model is: 18.593240727991528
Creating model
Loading data for dataset 1
Training model
Training time 5.876110216466145
Assesing model performance
100/100 [==============================] - 0s 4ms/step
The score for this model is: 17.365127643672956
Creating model
Loading data for dataset 1
Training model
Training time 37.10598978682526
Assesing model performance
100/100 [==============================] - 0s 4ms/step
The score for this model is: 20.788813473236033
Creating model
Loading data for dataset 1
Training model
Training time 38.15976901499789
Assesing model performance
100/100 [==============================] - 0s 4ms

Training model
Training time 26.17034302760044
Assesing model performance
100/100 [==============================] - 1s 11ms/step
The score for this model is: 16.47069472700041
Creating model
Loading data for dataset 1
Training model
Training time 12.01533292779709
Assesing model performance
100/100 [==============================] - 1s 11ms/step
The score for this model is: 13.405718127105985
Creating model
Loading data for dataset 1
Training model
Training time 9.624159558607516
Assesing model performance
100/100 [==============================] - 1s 12ms/step
The score for this model is: 17.868791770285814
Creating model
Loading data for dataset 1
Training model
Training time 52.493851005652914
Assesing model performance
100/100 [==============================] - 1s 12ms/step
The score for this model is: 15.77246779538463
Creating model
Loading data for dataset 1
Training model
Training time 9.506522510310106
Assesing model performance
100/100 [==============================] - 1s 1

100/100 [==============================] - 2s 15ms/step
The score for this model is: 13.703190319292723
Creating model
Loading data for dataset 1
Training model
Training time 13.774366857529003
Assesing model performance
100/100 [==============================] - 2s 15ms/step
The score for this model is: 14.851492553096726
Creating model
Loading data for dataset 1
Training model
Training time 11.296227659442138
Assesing model performance
100/100 [==============================] - 2s 16ms/step
The score for this model is: 16.451978458983096
Creating model
Loading data for dataset 1
Training model
Training time 11.166067342587212
Assesing model performance
100/100 [==============================] - 2s 16ms/step
The score for this model is: 13.577784671008214
Creating model
Loading data for dataset 1
Training model
Training time 14.23157160742312
Assesing model performance
100/100 [==============================] - 2s 16ms/step
The score for this model is: 14.806140821641318
Creating mode

100/100 [==============================] - 2s 20ms/step
The score for this model is: 15.598315886533832
Creating model
Loading data for dataset 1
Training model
Training time 16.308053145567555
Assesing model performance
100/100 [==============================] - 2s 20ms/step
The score for this model is: 14.801787211789188
Creating model
Loading data for dataset 1
Training model
Training time 17.04847832473706
Assesing model performance
100/100 [==============================] - 2s 20ms/step
The score for this model is: 15.591524002724656
Creating model
Loading data for dataset 1
Training model
Training time 37.188471473693426
Assesing model performance
100/100 [==============================] - 2s 20ms/step
The score for this model is: 19.620296292648796
Creating model
Loading data for dataset 1
Training model
Training time 13.199154225277198
Assesing model performance
100/100 [==============================] - 2s 20ms/step
The score for this model is: 14.861301403946603
Creating mode

...

Training time 63.33607196277444
Assesing model performance
100/100 [==============================] - 14s 139ms/step
The score for this model is: 13.181174760714985
Creating model
Loading data for dataset 1
Training model
Training time 64.53152930649958
Assesing model performance
100/100 [==============================] - 14s 139ms/step
The score for this model is: 12.825228528817966
Creating model
Loading data for dataset 1
Training model
Training time 84.88049829760712
Assesing model performance
100/100 [==============================] - 14s 141ms/step
The score for this model is: 12.372626762550105
Creating model
Loading data for dataset 1
Training model
Training time 59.56974862034258
Assesing model performance
100/100 [==============================] - 14s 141ms/step
The score for this model is: 12.947564020405538
Creating model
Loading data for dataset 1
Training model
Training time 59.536041242790816
Assesing model performance
100/100 [==============================] - 14s 141ms

Training time 61.49691934664588
Assesing model performance
100/100 [==============================] - 15s 152ms/step
The score for this model is: 12.975357136010047
Creating model
Loading data for dataset 1
Training model
Training time 63.440623030473944
Assesing model performance
100/100 [==============================] - 15s 152ms/step
The score for this model is: 12.360357770463894
Creating model
Loading data for dataset 1
Training model
Training time 88.61539363632619
Assesing model performance
100/100 [==============================] - 15s 152ms/step
The score for this model is: 12.166387663128742
Creating model
Loading data for dataset 1
Training model
Training time 72.89122347201919
Assesing model performance
100/100 [==============================] - 15s 152ms/step
The score for this model is: 13.071414188850584
Creating model
Loading data for dataset 1
Training model
Training time 54.26229670870089
Assesing model performance
100/100 [==============================] - 15s 152ms

Training time 67.57252463389159
Assesing model performance
100/100 [==============================] - 17s 171ms/step
The score for this model is: 12.781020398304472
Creating model
Loading data for dataset 1
Training model
Training time 71.42610076885467
Assesing model performance
100/100 [==============================] - 16s 165ms/step
The score for this model is: 11.667037947786818
Creating model
Loading data for dataset 1
Training model
Training time 78.36295926067396
Assesing model performance
100/100 [==============================] - 16s 163ms/step
The score for this model is: 13.39927658291882
differential_evolution step 9: f(x)= 11.2702
Creating model
Loading data for dataset 1
Training model
Training time 69.49704515227495
Assesing model performance
100/100 [==============================] - 16s 164ms/step
The score for this model is: 12.168024239941692
Creating model
Loading data for dataset 1
Training model
Training time 195.38955813207576
Assesing model performance
100/100 

Training model
Training time 73.56837499832181
Assesing model performance
100/100 [==============================] - 18s 178ms/step
The score for this model is: 11.644278847954304
Creating model
Loading data for dataset 1
Training model
Training time 122.41647078614915
Assesing model performance
100/100 [==============================] - 18s 180ms/step
The score for this model is: 13.31236284498742
Creating model
Loading data for dataset 1
Training model
Training time 86.54794474792288
Assesing model performance
100/100 [==============================] - 18s 180ms/step
The score for this model is: 13.70168079173335
Creating model
Loading data for dataset 1
Training model
Training time 67.02992369985441
Assesing model performance
100/100 [==============================] - 18s 179ms/step
The score for this model is: 12.354763678309471
Creating model
Loading data for dataset 1
Training model
Training time 63.398586637056724
Assesing model performance
100/100 [=============================

Training model
Training time 86.40328630657314
Assesing model performance
100/100 [==============================] - 19s 192ms/step
The score for this model is: 12.912253753588887
Creating model
Loading data for dataset 1
Training model
Training time 72.3097286035918
Assesing model performance
100/100 [==============================] - 19s 193ms/step
The score for this model is: 12.419522920005699
Creating model
Loading data for dataset 1
Training model
Training time 81.99600981576805
Assesing model performance
100/100 [==============================] - 19s 193ms/step
The score for this model is: 13.307015230994145
Creating model
Loading data for dataset 1
Training model
Training time 83.6865181948815
Assesing model performance
100/100 [==============================] - 19s 194ms/step


In [ ]:
print(results)